In [1]:
import sqlite3
import json
import os
import logging
import pandas as pd
from datetime import datetime
from io import BytesIO
from great_tables import GT, loc, style, px, html
from great_tables.data import islands
from pathlib import Path
from typing import Any

db_path = "database/reporting.db"
conn = sqlite3.connect(db_path)
cur = conn.cursor()
db_path = "database/reporting.db"
cutoff = pd.to_datetime("2025-04-15")

# Fetch latest table data
def fetch_latest_table_data(conn: sqlite3.Connection, table_alias: str, cutoff: pd.Timestamp) -> pd.DataFrame:
    cutoff_str = cutoff.isoformat()
    query = """
        SELECT uploaded_at, id
        FROM upload_log
        WHERE table_alias = ?
        ORDER BY ABS(strftime('%s', uploaded_at) - strftime('%s', ?))
        LIMIT 1
    """
    result = conn.execute(query, (table_alias, cutoff_str)).fetchone()
    if not result:
        raise ValueError(f"No uploads found for table alias '{table_alias}' near cutoff {cutoff_str}")
    
    closest_uploaded_at, upload_id = result
    df = pd.read_sql_query(
        f"SELECT * FROM {table_alias} WHERE upload_id = ?",
        conn,
        params=(upload_id,)
    )
    return df

# Main execution
try:
    conn = sqlite3.connect(db_path)
    df = fetch_latest_table_data(conn, "c0_budgetary_execution_details", cutoff)
except Exception as e:
    print("Error:", e)
    logging.error(f"Main execution error: {str(e)}")
finally:
    conn.close()

df.columns

Index(['Financial Management Area', 'FMA Desc', 'Fiscal Year',
       'Administrative/Operational Flg', 'Budget Address',
       'Official Budget Line', 'Fund Source', 'Fund Source Desc',
       'Budget Period', 'Commitment Item', 'Commitment Item Desc',
       'Functional Area', 'Functional Area Desc', 'Funded Program',
       'Funded Program Desc', 'Commitment Appropriation', 'Committed Amount',
       'Commitment Available ', '%', 'Payment Appropriation', 'Paid Amount',
       'Payment Available', '%.1', 'upload_id', 'uploaded_at'],
      dtype='object')

In [2]:
import sqlite3
import json
import os
import logging
import pandas as pd
from datetime import datetime
from io import BytesIO
from great_tables import GT, loc, style, px, html
from great_tables.data import islands
from pathlib import Path
from typing import Any

# Define colors
BLUE        = "#004A99"
LIGHT_BLUE = "#d6e6f4"
GRID_CLR    = "#004A99"
DARK_BLUE   = "#01244B"
DARK_GREY =   '#242425'


# Parameters
db_path = "database/reporting.db"
cutoff = pd.to_datetime("2025-04-15")
current_year = 2025
report = "Quarterly_Report"

# Configure logging
logging.basicConfig(level=logging.DEBUG)

def insert_variable(
    report: str,
    module: str,
    var: str,
    value: Any,
    db_path: str,
    anchor: str | None = None,
    gt_table=None,
) -> None:
    """
    Overwrite the row (report_name, var_name) with a new value (and picture).

    Exactly ONE row per variable is kept.
    """
    con = sqlite3.connect(db_path)
    cur = con.cursor()
    try:
        # 1) remove any previous copy of this variable
        cur.execute(
            "DELETE FROM report_variables WHERE report_name = ? AND var_name = ?",
            (report, var),
        )

        # 2) serialise the Python value
        val_json = json.dumps(value, default=str)

        # 3) optional: render great‑tables object to PNG → bytes
        gt_image = None
        if gt_table is not None:
            tmp = Path(f"__gt_{var}.png")
            gt_table.save(tmp)  # playwright renders PNG
            gt_image = tmp.read_bytes()
            tmp.unlink(missing_ok=True)

        # 4) insert the fresh row
        cur.execute(
            """
            INSERT INTO report_variables
                  (report_name, module_name, var_name,
                   anchor_name, value, gt_image, created_at)
            VALUES (?, ?, ?, ?, ?, ?, CURRENT_TIMESTAMP)
            """,
            (report, module, var, anchor or var, val_json, gt_image),
        )

        con.commit()
        logging.debug("Stored variable %s for report %s (rowid=%s)",
                      var, report, cur.lastrowid)

    except Exception as exc:
        con.rollback()
        logging.error("insert_variable failed for %s/%s: %s", report, var, exc)
        raise
    finally:
        con.close()

# Fetch latest table data
def fetch_latest_table_data(conn: sqlite3.Connection, table_alias: str, cutoff: pd.Timestamp) -> pd.DataFrame:
    cutoff_str = cutoff.isoformat()
    query = """
        SELECT uploaded_at, id
        FROM upload_log
        WHERE table_alias = ?
        ORDER BY ABS(strftime('%s', uploaded_at) - strftime('%s', ?))
        LIMIT 1
    """
    result = conn.execute(query, (table_alias, cutoff_str)).fetchone()
    if not result:
        raise ValueError(f"No uploads found for table alias '{table_alias}' near cutoff {cutoff_str}")
    closest_uploaded_at, upload_id = result
    df = pd.read_sql_query(
        f"SELECT * FROM {table_alias} WHERE upload_id = ?",
        conn,
        params=(upload_id,)
    )
    return df

def build_commitment_summary_table(df: pd.DataFrame, current_year: int, report: str, db_path: str) -> pd.DataFrame:
    """
    Builds a commitment summary table for the HE programme, grouped by Fund Source and Budget Address Type.
    
    Args:
        df (pd.DataFrame): Input DataFrame with commitment data.
        current_year (int): The year to filter the Budget Period.
        report (str): Name of the report for storing the table.
        db_path (str): Path to the SQLite database for storing the table.
    
    Returns:
        pd.DataFrame: Aggregated table with commitment summary.
    """
    # Define expected columns for validation
    expected_columns = [
        "Budget Period", "Fund Source", "Functional Area Desc", 
        "Budget Address", "Commitment Appropriation", 
        "Committed Amount", "Commitment Available "
    ]
    
    # Validate input DataFrame
    if not isinstance(df, pd.DataFrame):
        logging.error("Input 'df' is not a pandas DataFrame.")
        return pd.DataFrame(columns=[
            "Fund Source", "Budget Address Type", "Available_Commitment_Appropriations",
            "L1_Commitment", "RAC_on_Appropriation", 
            "ratio_consumed_of_L1_and_L2_against_Commitment_Appropriations"
        ])
    
    if df.empty:
        logging.warning("Input DataFrame is empty. Returning empty DataFrame with expected columns.")
        return pd.DataFrame(columns=[
            "Fund Source", "Budget Address Type", "Available_Commitment_Appropriations",
            "L1_Commitment", "RAC_on_Appropriation", 
            "ratio_consumed_of_L1_and_L2_against_Commitment_Appropriations"
        ])
    
    # Check for missing columns
    missing_columns = [col for col in expected_columns if col not in df.columns]
    if missing_columns:
        logging.error(f"Missing required columns: {missing_columns}")
        return pd.DataFrame(columns=[
            "Fund Source", "Budget Address Type", "Available_Commitment_Appropriations",
            "L1_Commitment", "RAC_on_Appropriation", 
            "ratio_consumed_of_L1_and_L2_against_Commitment_Appropriations"
        ])
    
    # Log input DataFrame info
    logging.debug(f"Input DataFrame shape: {df.shape}")
    logging.debug(f"Input DataFrame columns: {df.columns.tolist()}")
    
    try:
        # Filter for current year and specific Fund Sources
        df = df[df["Budget Period"] == current_year].copy()
        logging.debug(f"After filtering Budget Period == {current_year}: {df.shape[0]} rows")
        
        df = df[df["Fund Source"].isin(["VOBU", "EFTA"])].copy()
        logging.debug(f"After filtering Fund Source in ['VOBU', 'EFTA']: {df.shape[0]} rows")
        
        # Map Functional Area to Programme
        df.loc[:, "Programme"] = df["Functional Area Desc"].replace({
            "HORIZONEU_21_27": "HE",
            "H2020_14_20": "H2020"
        })
        
        # Filter for HE programme
        df = df.loc[df["Programme"] == "HE"].copy()
        logging.debug(f"After filtering Programme == 'HE': {df.shape[0]} rows")
        
        if df.empty:
            logging.warning("DataFrame is empty after filtering. Returning empty DataFrame.")
            return pd.DataFrame(columns=[
                "Fund Source", "Budget Address Type", "Available_Commitment_Appropriations",
                "L1_Commitment", "RAC_on_Appropriation", 
                "ratio_consumed_of_L1_and_L2_against_Commitment_Appropriations"
            ])
        
        # Budget-type mapping helper
        def map_budget_type(val):
            if pd.isna(val):
                return None
            v = str(val).upper()
            if "EMPTY" in v:
                return "Main Calls"
            if "EXPERTS" in v:
                return "Experts"
            return val
        
        df.loc[:, "Budget_Address_Type"] = df["Budget Address"].apply(map_budget_type)
        logging.debug(f"Unique Budget_Address_Type values: {df['Budget_Address_Type'].unique()}")
        
        # Fund-type mapping helper
        def map_fund_type(val):
            if pd.isna(val):
                return None
            v = str(val).upper()
            if "VOBU" in v:
                return "VOBU/EFTA"
            if "EFTA" in v:
                return "VOBU/EFTA"
            if "IAR2/2" in v:
                return "VOBU/EFTA"
            return val
        
        df['Fund Source'] = df['Fund Source'].apply(map_fund_type)
        logging.debug(f"Unique Fund Source values after mapping: {df['Fund Source'].unique()}")
        
        # Group by Fund Source and Budget_Address_Type
        agg = df.groupby(["Fund Source", "Budget_Address_Type"])[
            ["Commitment Appropriation", "Committed Amount", "Commitment Available "]
        ].sum().reset_index()
        logging.debug(f"Aggregated DataFrame shape: {agg.shape}")
        
        # Compute percentage
        agg["%"] = agg["Committed Amount"] / agg["Commitment Appropriation"].replace(0, pd.NA)
        agg["%"] = agg["%"].fillna(0)  # Handle division by zero
        
        # Rename columns
        agg = agg.rename(columns={
            "Commitment Appropriation": "Available_Commitment_Appropriations",
            "Committed Amount": "L1_Commitment",
            "Commitment Available ": "RAC_on_Appropriation",
            "%": "ratio_consumed_of_L1_and_L2_against_Commitment_Appropriations",
            "Budget_Address_Type": "Budget Address Type"
        })
        
        # Add total row
        try:
            total_row = pd.DataFrame({
                "Fund Source": ["VOBU/EFTA"],
                "Budget Address Type": ["Total"],
                "Available_Commitment_Appropriations": [agg["Available_Commitment_Appropriations"].sum()],
                "L1_Commitment": [agg["L1_Commitment"].sum()],
                "RAC_on_Appropriation": [agg["RAC_on_Appropriation"].sum()],
                "ratio_consumed_of_L1_and_L2_against_Commitment_Appropriations": [
                    agg["L1_Commitment"].sum() / agg["Available_Commitment_Appropriations"].sum()
                    if agg["Available_Commitment_Appropriations"].sum() != 0 else 0
                ]
            })
            agg = pd.concat([agg, total_row], ignore_index=True)
            logging.debug(f"Final aggregated DataFrame with total row: {agg.shape}")
        except Exception as e:
            logging.error(f"Error adding total row: {str(e)}")
            return agg  # Proceed without total row if it fails
        
        # Build GreatTables object
        try:
            tbl = (
                GT(
                    agg,
                    rowname_col="Budget Address Type",
                    groupname_col="Fund Source"
                )
                .tab_header("HE")
                .tab_stubhead(label="Budget Address Type")
                # Formats
                .fmt_number(columns=[
                    "Available_Commitment_Appropriations",
                    "L1_Commitment",
                    "RAC_on_Appropriation"
                ], accounting=True, decimals=2)
                .fmt_percent(
                    columns="ratio_consumed_of_L1_and_L2_against_Commitment_Appropriations",
                    decimals=2
                )
                # Custom column labels
                .cols_label(
                    Available_Commitment_Appropriations=html("Available Commitment Appropriations<br>(1)"),
                    L1_Commitment=html("L1 Commitments or <br> Direct L2 <br>(2)"),
                    RAC_on_Appropriation=html("RAC on Appropriation<br>(3)=(1)-(2)"),
                    ratio_consumed_of_L1_and_L2_against_Commitment_Appropriations=html(
                        "% Commitment Appropriations by <br> L1 and Direct L2 Commitments <br> (4) = (2)/(1)"
                    )
                )
                # Arial font
                .opt_table_font(font="Arial")
                # Header and stub styling
                .tab_style(
                    style=[
                        style.text(color=DARK_BLUE, weight="bold"),
                        style.fill(color=LIGHT_BLUE),
                        style.css(f"border-bottom: 2px solid {DARK_BLUE}; border-right: 2px solid {DARK_BLUE}; border-top: 2px solid {DARK_BLUE}; border-left: 2px solid {DARK_BLUE};"),
                        style.css("max-width:200px; line-height:1.2"),
                    ],
                    locations=loc.row_groups()
                )
                .tab_style(
                    style=[
                        style.fill(color=BLUE),
                        style.text(color="white", weight="bold", align='center'),
                        style.css("text-align: center; vertical-align: middle; max-width:200px; line-height:1.2")
                    ],
                    locations=loc.column_labels()
                )
                .tab_style(
                    style=[
                        style.fill(color=BLUE),
                        style.text(color="white", weight="bold"),
                        style.css("text-align: center; vertical-align: middle; max-width:200px; line-height:1.2")
                    ],
                    locations=loc.stubhead()
                )
                .tab_style(
                    style=[
                        style.text(color=DARK_BLUE, weight="bold"),
                    ],
                    locations=loc.header()
                )
                # Grid lines
                .tab_style(
                    style=style.borders(weight="1px", color=DARK_BLUE),
                    locations=loc.stub()
                )
                .tab_style(
                    style=style.borders(sides=["all"], color=DARK_BLUE, weight='2px'),
                    locations=loc.body()
                )
                .tab_style(
                    style=style.borders(color=DARK_BLUE, weight='2px'),
                    locations=loc.column_labels()
                )
                .tab_style(
                    style=style.borders(color=DARK_BLUE, weight='2px'),
                    locations=loc.stubhead()
                )
                # Style total row
                .tab_style(
                    style=[style.fill(color="#E6E6FA"), style.text(color="black", weight="bold")],
                    locations=loc.body(rows=agg.index[-1])
                )
                .tab_style(
                    style=[style.fill(color="#E6E6FA"), style.text(weight="bold")],
                    locations=loc.stub(rows=[-1])
                )
                # Table borders
                .tab_options(table_body_border_bottom_color=DARK_BLUE, table_body_border_bottom_width="2px")
                .tab_options(table_border_right_color=DARK_BLUE, table_border_right_width="2px")
                .tab_options(table_border_left_color=DARK_BLUE, table_border_left_width="2px")
                .tab_options(table_border_top_color=DARK_BLUE, table_border_top_width="2px")
                .tab_options(column_labels_border_top_color=DARK_BLUE, column_labels_border_top_width="2px")
                # Source notes
                .tab_source_note("Source: Summa DataWarehouse")
                .tab_source_note("BO Report: C0_Budgetary_Execution_Details")
            )
        except Exception as e:
            logging.error(f"Error building GreatTables object: {str(e)}")
            # Return the aggregated DataFrame without styling if table creation fails
            return agg
        
        # Store the table
        try:
            insert_variable(
                report=report,
                module="BudgetModule",
                var="table_1a_commitment_summary",
                value=agg.to_dict(orient="records"),
                db_path=db_path,
                anchor="table_1a",
                gt_table=tbl
            )
            logging.debug("Successfully stored table_1a_commitment_summary")
        except Exception as e:
            logging.error(f"Error storing table: {str(e)}")
            # Continue even if storage fails, as the table is already built
        
        return agg
    
    except Exception as e:
        logging.error(f"Unexpected error in build_commitment_summary_table: {str(e)}")
        return pd.DataFrame(columns=[
            "Fund Source", "Budget Address Type", "Available_Commitment_Appropriations",
            "L1_Commitment", "RAC_on_Appropriation", 
            "ratio_consumed_of_L1_and_L2_against_Commitment_Appropriations"
        ])

# Main execution
try:
    conn = sqlite3.connect(db_path)
    df_comm = fetch_latest_table_data(conn, "c0_budgetary_execution_details", cutoff)
    agg = build_commitment_summary_table(df_comm, current_year, report, db_path)
except Exception as e:
    print("Error:", e)
    logging.error(f"Main execution error: {str(e)}")
finally:
    conn.close()

DEBUG:root:Input DataFrame shape: (43, 25)
DEBUG:root:Input DataFrame columns: ['Financial Management Area', 'FMA Desc', 'Fiscal Year', 'Administrative/Operational Flg', 'Budget Address', 'Official Budget Line', 'Fund Source', 'Fund Source Desc', 'Budget Period', 'Commitment Item', 'Commitment Item Desc', 'Functional Area', 'Functional Area Desc', 'Funded Program', 'Funded Program Desc', 'Commitment Appropriation', 'Committed Amount', 'Commitment Available ', '%', 'Payment Appropriation', 'Paid Amount', 'Payment Available', '%.1', 'upload_id', 'uploaded_at']
DEBUG:root:After filtering Budget Period == 2025: 7 rows
DEBUG:root:After filtering Fund Source in ['VOBU', 'EFTA']: 6 rows
DEBUG:root:After filtering Programme == 'HE': 4 rows
DEBUG:root:Unique Budget_Address_Type values: ['Main Calls' 'Experts']
DEBUG:root:Unique Fund Source values after mapping: ['VOBU/EFTA']
DEBUG:root:Aggregated DataFrame shape: (2, 5)
DEBUG:root:Final aggregated DataFrame with total row: (3, 6)
DEBUG:selenium

In [4]:
tbl 

NameError: name 'tbl' is not defined

In [5]:
import sqlite3
import json
import os
import logging
import pandas as pd
from datetime import datetime
from io import BytesIO
from great_tables import GT, loc, style, px, html
from great_tables.data import islands
from pathlib import Path
from typing import Any

# Define colors
BLUE = "#004A99"
LIGHT_BLUE = "#579DEE"
GRID_CLR = "#004A99"
DARK_BLUE = "#01244B"
DARK_GREY = '#242425'

# Parameters
db_path = "database/reporting.db"
cutoff = pd.to_datetime("2025-04-15")
current_year = 2025
report = "Quarterly_Report"

# Configure logging
logging.basicConfig(level=logging.DEBUG)

def insert_variable(
    report: str,
    module: str,
    var: str,
    value: Any,
    db_path: str,
    anchor: str | None = None,
    gt_table=None,
) -> None:
    """
    Overwrite the row (report_name, var_name) with a new value (and picture).

    Exactly ONE row per variable is kept.
    """
    con = sqlite3.connect(db_path)
    cur = con.cursor()
    try:
        # 1) remove any previous copy of this variable
        cur.execute(
            "DELETE FROM report_variables WHERE report_name = ? AND var_name = ?",
            (report, var),
        )

        # 2) serialise the Python value
        val_json = json.dumps(value, default=str)

        # 3) optional: render great‑tables object to PNG → bytes
        gt_image = None
        if gt_table is not None:
            tmp = Path(f"__gt_{var}.png")
            gt_table.save(tmp)  # playwright renders PNG
            gt_image = tmp.read_bytes()
            tmp.unlink(missing_ok=True)

        # 4) insert the fresh row
        cur.execute(
            """
            INSERT INTO report_variables
                  (report_name, module_name, var_name,
                   anchor_name, value, gt_image, created_at)
            VALUES (?, ?, ?, ?, ?, ?, CURRENT_TIMESTAMP)
            """,
            (report, module, var, anchor or var, val_json, gt_image),
        )

        con.commit()
        logging.debug("Stored variable %s for report %s (rowid=%s)",
                      var, report, cur.lastrowid)

    except Exception as exc:
        con.rollback()
        logging.error("insert_variable failed for %s/%s: %s", report, var, exc)
        raise
    finally:
        con.close()

# Fetch latest table data
def fetch_latest_table_data(conn: sqlite3.Connection, table_alias: str, cutoff: pd.Timestamp) -> pd.DataFrame:
    cutoff_str = cutoff.isoformat()
    query = """
        SELECT uploaded_at, id
        FROM upload_log
        WHERE table_alias = ?
        ORDER BY ABS(strftime('%s', uploaded_at) - strftime('%s', ?))
        LIMIT 1
    """
    result = conn.execute(query, (table_alias, cutoff_str)).fetchone()
    if not result:
        raise ValueError(f"No uploads found for table alias '{table_alias}' near cutoff {cutoff_str}")
    closest_uploaded_at, upload_id = result
    df = pd.read_sql_query(
        f"SELECT * FROM {table_alias} WHERE upload_id = ?",
        conn,
        params=(upload_id,)
    )
    return df

def build_payment_summary_tables(
        df: pd.DataFrame,
        current_year: int,
        report: str,
        db_path: str
) -> dict[str, pd.DataFrame]:
    """
    Build & store one payment‑summary table per programme (HE, H2020).

    Returns
    -------
    dict
        keys  = programme code ("HE", "H2020")
        value = aggregated DataFrame for that programme
    """
    # ---------- common pre‑filtering ---------------------------------------------------
    # Create a new DataFrame with a copy to avoid SettingWithCopyWarning
    df = df[(df["Budget Period"] == current_year) & (df["Fund Source"].isin(["VOBU", "EFTA"]))].copy()

    # Map Functional Area → Programme
    df.loc[:, "Programme"] = df["Functional Area Desc"].replace({
        "HORIZONEU_21_27": "HE",
        "H2020_14_20": "H2020"
    })

    # Budget‑type mapping helper
    def map_budget_type(val):
        if pd.isna(val):
            return None
        v = str(val).upper()
        if "EMPTY" in v: return "Main Call"
        if "EXPERTS" in v: return "Experts"
        return val

    df.loc[:, "Budget_Address_Type"] = df["Budget Address"].apply(map_budget_type)

    # Set Fund Source for all rows
    df.loc[:, "Fund Source"] = 'VOBU/EFTA'

    # ---------- iterate over the two programmes ---------------------------------------
    results: dict[str, pd.DataFrame] = {}
    for programme in ("HE", "H2020"):
        df_p = df[df["Programme"] == programme].copy()

        if df_p.empty:
            logging.warning("No rows for programme %s", programme)
            continue

        # -------- aggregation ------------------------------------------------------
        agg = (df_p
               .groupby(["Fund Source", "Budget_Address_Type"],
                        as_index=False)[
                    ["Payment Appropriation", "Paid Amount",
                     "Payment Available"]]
               .sum())

        agg["%"] = agg["Paid Amount"] / agg["Payment Appropriation"]

        agg = agg.rename(columns={
            "Payment Appropriation": "Available_Payment_Appropriations",
            "Paid Amount": "Paid_Amount",
            "Payment Available": "Remaining_Payment_Appropriation",
            "%": "ratio_consumed_Payment_Appropriations",
            "Budget_Address_Type": "Budget Address Type"
        })

        # -------- Add total row ---------------------------------------------------
        total_row = pd.DataFrame({
            # "Programme": ["Total"],
            "Fund Source": ['VOBU/EFTA'],
            "Budget Address Type": ['Total'],
            "Available_Payment_Appropriations": [agg["Available_Payment_Appropriations"].sum()],
            "Paid_Amount": [agg["Paid_Amount"].sum()],
            "Remaining_Payment_Appropriation": [agg["Remaining_Payment_Appropriation"].sum()],
            "ratio_consumed_Payment_Appropriations": [agg["Paid_Amount"].sum() / agg["Available_Payment_Appropriations"].sum()]
        })
        agg = pd.concat([agg, total_row], ignore_index=True)

        # -------- GreatTables object -------------------------------------------
        tbl = (
            GT(agg,
              rowname_col="Budget Address Type",
              groupname_col="Fund Source"
              )
            .tab_stubhead(label="Budget Address Type")
            .tab_style(
                style=[
                    style.text(color="white", weight="bold"),
                    style.fill(color=LIGHT_BLUE),
                    style.css(f"border-bottom: 2px solid {DARK_BLUE}; border-right: 2px solid {DARK_BLUE}; border-top: 2px solid {DARK_BLUE}; border-left: 2px solid {DARK_BLUE};"),
                    style.css("max-width:200px; line-height:1.2"),
                ],
                locations=loc.row_groups()
            )
            .tab_header(
                title=f"{programme}",
            )
            .fmt_number(columns=[
                "Available_Payment_Appropriations",
                "Paid_Amount",
                "Remaining_Payment_Appropriation"
            ], accounting=True, decimals=2)
            .fmt_percent(
                columns="ratio_consumed_Payment_Appropriations",
                decimals=2)
            .cols_label(
                Available_Payment_Appropriations=html("Payment Appropriations<br>(1)"),
                Paid_Amount=html("Payment Credits consumed<br>(Acceptance Date)<br>(2)"),
                Remaining_Payment_Appropriation=html("Remaining Payment Appropriations<br>(3)=(1)-(2)"),
                ratio_consumed_Payment_Appropriations=html("% Payment Consumed<br>(4) = (2)/(1)")
            )
            .opt_table_font(font="Arial")

              .tab_style(
                style = [style.text(weight="bold", color=DARK_BLUE)],
                locations= loc.header(),
                )

            .tab_style(
            style=[style.fill(color=BLUE),
                   style.text(color="white", weight="bold", align="center"),
                   style.css("text-align: center;vertical-align: middle; max-width:200px; line-height:1.2")],
            locations=loc.column_labels())
            .tab_style(
                style=[style.fill(color=BLUE),
                    style.text(color="white", weight="bold"),
                    style.css("text-align: center;vertical-align: middle; max-width:200px; line-height:1.2")],
                locations=loc.stubhead())

            .tab_style(
            style.borders(weight="1px", color=DARK_BLUE),
            loc.stub(),
            )

            .tab_style(
                style=style.borders(sides="all", color=DARK_BLUE, weight="1px"),
                locations=loc.body())
            .tab_style(
                style=style.borders(color=DARK_BLUE, weight="2px"),
                locations=[loc.column_labels(), loc.stubhead()])

            .tab_style(
                style=[style.fill(color="#E6E6FA"), style.text(color="black", weight="bold")],
                locations=loc.body(rows=agg.index[-1]))  # Apply to the last row (total)

             .tab_style(
                style = [style.fill( color="#E6E6FA"), style.text(weight="bold")],
                locations= loc.stub(rows=[-1]),
                )

            .tab_options(table_body_border_bottom_color=DARK_BLUE,
                     table_body_border_bottom_width="2px")
            .tab_options(table_border_right_color=DARK_BLUE,
                        table_border_right_width="2px")
            .tab_options(table_border_left_color=DARK_BLUE,
                        table_border_left_width="2px")
            .tab_options(table_border_top_color=DARK_BLUE,
                        table_border_top_width="2px")
            .tab_options(column_labels_border_top_color=DARK_BLUE,
                        column_labels_border_top_width="2px")
            .tab_source_note("Source: Summa DataWarehouse")
            .tab_source_note("BO Report: C0_Budgetary_Execution_Details")
        )

        # -------- store --------------------------------------------------------
        insert_variable(
            report=report,
            module="BudgetModule",
            var=f"table_2a_{programme}_data",
            value=agg.to_dict(orient="records"),
            db_path=db_path,
            anchor=f"table_2a_{programme}",
            gt_table=tbl,
        )

        logging.debug("Stored 2a table for programme %s (%d rows)",
                      programme, len(agg))

        results[programme] = agg

    #---- END loop ------
    return results

# Main execution
try:
    conn = sqlite3.connect(db_path)
    df = fetch_latest_table_data(conn, "c0_budgetary_execution_details", cutoff)
    results = build_payment_summary_tables(df, current_year, report, db_path)  # Updated to match return type
except Exception as e:
    print("Error:", e)
    logging.error(f"Main execution error: {str(e)}")
finally:
    conn.close()

DEBUG:selenium.webdriver.common.selenium_manager:Selenium Manager binary found at: c:\Users\vinci\anaconda3\envs\qenv\Lib\site-packages\selenium\webdriver\common\windows\selenium-manager.exe
DEBUG:selenium.webdriver.common.selenium_manager:Executing process: c:\Users\vinci\anaconda3\envs\qenv\Lib\site-packages\selenium\webdriver\common\windows\selenium-manager.exe --browser chrome --debug --language-binding python --output json
DEBUG:selenium.webdriver.common.selenium_manager:Found chromedriver 138.0.7204.92 in PATH: f:\vinci\Projects\QuarterlyReport\chromedriver.exe
DEBUG:selenium.webdriver.common.selenium_manager:chrome detected at C:\Program Files\Google\Chrome\Application\chrome.exe
DEBUG:selenium.webdriver.common.selenium_manager:Running command: wmic datafile where name='C:\\Program Files\\Google\\Chrome\\Application\\chrome.exe' get Version /value
DEBUG:selenium.webdriver.common.selenium_manager:Output: "\r\r\n\r\r\nVersion=138.0.7204.97\r\r\n\r\r\n\r\r\n\r"
DEBUG:selenium.webdr

In [6]:
import sqlite3
import json
import os
import logging
import pandas as pd
from datetime import datetime
from io import BytesIO
from great_tables import GT, loc, style, px, html
from great_tables.data import islands
from pathlib import Path
from typing import Any

BLUE        = "#004A99"
LIGHT_BLUE = "#d6e6f4"
GRID_CLR    = "#004A99"
DARK_BLUE   = "#01244B"
DARK_GREY =   '#242425'


# Parameters
db_path = "database/reporting.db"
cutoff = pd.to_datetime("2025-04-15")
current_year = 2025
report = "Quarterly_Report"

# Configure logging
logging.basicConfig(level=logging.DEBUG)

def insert_variable(
    report: str,
    module: str,
    var: str,
    value: Any,
    db_path: str,
    anchor: str | None = None,
    gt_table=None,
) -> None:
    """
    Overwrite the row (report_name, var_name) with a new value (and picture).

    Exactly ONE row per variable is kept.
    """
    con = sqlite3.connect(db_path)
    cur = con.cursor()
    try:
        # 1) remove any previous copy of this variable
        cur.execute(
            "DELETE FROM report_variables WHERE report_name = ? AND var_name = ?",
            (report, var),
        )

        # 2) serialise the Python value
        val_json = json.dumps(value, default=str)

        # 3) optional: render great‑tables object to PNG → bytes
        gt_image = None
        if gt_table is not None:
            tmp = Path(f"__gt_{var}.png")
            gt_table.save(tmp)  # playwright renders PNG
            gt_image = tmp.read_bytes()
            tmp.unlink(missing_ok=True)

        # 4) insert the fresh row
        cur.execute(
            """
            INSERT INTO report_variables
                  (report_name, module_name, var_name,
                   anchor_name, value, gt_image, created_at)
            VALUES (?, ?, ?, ?, ?, ?, CURRENT_TIMESTAMP)
            """,
            (report, module, var, anchor or var, val_json, gt_image),
        )

        con.commit()
        logging.debug("Stored variable %s for report %s (rowid=%s)",
                      var, report, cur.lastrowid)

    except Exception as exc:
        con.rollback()
        logging.error("insert_variable failed for %s/%s: %s", report, var, exc)
        raise
    finally:
        con.close()

# Fetch latest table data
def fetch_latest_table_data(conn: sqlite3.Connection, table_alias: str, cutoff: pd.Timestamp) -> pd.DataFrame:
    cutoff_str = cutoff.isoformat()
    query = """
        SELECT uploaded_at, id
        FROM upload_log
        WHERE table_alias = ?
        ORDER BY ABS(strftime('%s', uploaded_at) - strftime('%s', ?))
        LIMIT 1
    """
    result = conn.execute(query, (table_alias, cutoff_str)).fetchone()
    if not result:
        raise ValueError(f"No uploads found for table alias '{table_alias}' near cutoff {cutoff_str}")
    closest_uploaded_at, upload_id = result
    df = pd.read_sql_query(
        f"SELECT * FROM {table_alias} WHERE upload_id = ?",
        conn,
        params=(upload_id,)
    )
    return df

def build_commitment_detail_table_2(df: pd.DataFrame, current_year: int, report: str, db_path: str) -> pd.DataFrame:
    df["FR ILC Date (dd/mm/yyyy)"] = pd.to_datetime(df["FR ILC Date (dd/mm/yyyy)"], errors="coerce")
    eoy_this = pd.Timestamp(f"{current_year}-12-31")

    # Budget‑type mapping helper
    def map_budget_type(val):
        if pd.isna(val):
            return None
        v = str(val).upper()
        if "VOBU" in v: return "VOBU/EFTA/IAR2/2"
        if "EFTA" in v: return "VOBU/EFTA/IAR2/2"
        if "IAR2/2" in v: return "VOBU/EFTA/IAR2/2"
        return val

    df['Fund Source'] = df['Fund Source'].apply(map_budget_type)

    filtered = df[(df["FR Earmarked Document Type Desc"] == "Global Commitment") &
                  (df["FR ILC Date (dd/mm/yyyy)"] == eoy_this)].copy()

    filtered = filtered.rename(columns={
        "FR Accepted Amount": "L1_Commitment_1",
        "FR Consumption by PO Amount": "L2_Commitment_2"
    })

    agg = (filtered
           .groupby(["FR Fund Reservation Desc", "Fund Source"],
                    as_index=False)[
               ["L1_Commitment_1", "L2_Commitment_2"]]
           .sum())

    agg["RAC_on_L1_Commitment_3"] = agg["L1_Commitment_1"] + agg["L2_Commitment_2"]
    agg["ratio_L2_on_L1_Commitment_4"] = agg["L2_Commitment_2"] / (-1 * agg["L1_Commitment_1"])

    agg = agg[[
        "FR Fund Reservation Desc", "Fund Source", "L1_Commitment_1", "L2_Commitment_2",
        "RAC_on_L1_Commitment_3", "ratio_L2_on_L1_Commitment_4"
    ]]

    # -------- Add subtotals by Fund Source -----------------------------------
    # Group by Fund Source and compute subtotals
    subtotals = agg.groupby("Fund Source", as_index=False).agg({
        "L1_Commitment_1": "sum",
        "L2_Commitment_2": "sum",
        "RAC_on_L1_Commitment_3": "sum",
    })

    # Add the ratio for subtotals
    subtotals["ratio_L2_on_L1_Commitment_4"] = subtotals["L2_Commitment_2"] / (-1 * subtotals["L1_Commitment_1"])
    subtotals["FR Fund Reservation Desc"] = "Subtotal"

    # Concatenate original data with subtotals
    final_rows = []
    for fund_source in agg["Fund Source"].unique():
        # Rows for this Fund Source
        group_rows = agg[agg["Fund Source"] == fund_source].copy()
        final_rows.append(group_rows)
        # Corresponding subtotal row
        subtotal_row = subtotals[subtotals["Fund Source"] == fund_source].copy()
        final_rows.append(subtotal_row)

    # Combine all rows into the final DataFrame
    agg_with_subtotals = pd.concat(final_rows, ignore_index=True)

    # -------- GreatTables object -------------------------------------------
    tbl = (
        GT(
            agg_with_subtotals,
            rowname_col="FR Fund Reservation Desc",
            groupname_col="Fund Source"
        )
        .tab_stubhead(label="L1 Commitments")
        .tab_style(
            style=[
                style.text(color=DARK_BLUE, weight="bold", font='Arial'),
                style.fill(color=LIGHT_BLUE),
                style.css(f"border-bottom: 2px solid {DARK_BLUE}; border-right: 2px solid {DARK_BLUE}; border-top: 2px solid {DARK_BLUE}; border-left: 2px solid {DARK_BLUE};"),
                style.css("max-width:200px; line-height:1.2"),
            ],
            locations=loc.row_groups()
        )
        .fmt_number(columns=[
            "L1_Commitment_1",
            "L2_Commitment_2",
            "RAC_on_L1_Commitment_3"
        ], accounting=True, decimals=2)
        .fmt_percent(
            columns="ratio_L2_on_L1_Commitment_4",
            decimals=2)
        .cols_label(
            L1_Commitment_1=html("L1 Commitment<br>(1)"),
            L2_Commitment_2=html("L2 Commitment<br>(2)"),
            RAC_on_L1_Commitment_3=html("RAC on L1 Commitment <br>(3) = (1) - (2)"),
            ratio_L2_on_L1_Commitment_4=html("% L1 consumed by L2 (indirect)<br> (4) = (2) / (1)")
        )
        .opt_table_font(font="Arial")
        .tab_style(
                    style=[style.text(weight="bold", color=DARK_BLUE)],
                    locations=loc.header()
                    )
        .tab_style(
            style=[style.fill(color=BLUE),
                   style.text(color="white", weight="bold", align="center"),
                   style.css("max-width:200px; line-height:1.2")],
            locations=loc.column_labels())
        .tab_style(
            style=[style.fill(color=BLUE),
                   style.text(color="white", weight="bold"),
                   style.css("text-align: center;vertical-align: middle; max-width:200px; line-height:1.2")],
            locations=loc.stubhead())

        .tab_style(
        style.borders(weight="1px", color=DARK_BLUE),
        loc.stub(),
         )

        .tab_style(
            style=style.borders(sides="all", color=DARK_BLUE, weight="1px"),
            locations=loc.body())
        .tab_style(
            style=style.borders(color=DARK_BLUE, weight="2px"),
            locations=[loc.column_labels(), loc.stubhead()])
        .tab_style(
            style=[style.fill(color="#E6E6FA"), style.text(color="black", weight="bold")],
            locations=loc.body(rows=agg_with_subtotals.index[agg_with_subtotals["FR Fund Reservation Desc"] == "Subtotal"].tolist())
        )
        .tab_style(
                style=[style.fill(color="#E6E6FA"), style.text(weight="bold")],
                locations=loc.stub(rows=agg_with_subtotals.index[agg_with_subtotals["FR Fund Reservation Desc"] == "Subtotal"].tolist())
            )
        .tab_options(table_body_border_bottom_color=DARK_BLUE,
                     table_body_border_bottom_width="2px")
        .tab_options(table_border_right_color=DARK_BLUE,
                     table_border_right_width="2px")
        .tab_options(table_border_left_color=DARK_BLUE,
                     table_border_left_width="2px")
        .tab_options(table_border_top_color=DARK_BLUE,
                     table_border_top_width="2px")
        .tab_options(column_labels_border_top_color=DARK_BLUE,
                     column_labels_border_top_width="2px")

        .tab_source_note("Source: Summa DataWarehouse")
        .tab_source_note("BO Report: C0_COMMITMENTS_SUMMA")
    )

    # -------- store --------------------------------------------------------
    insert_variable(
        report=report,
        module="BudgetModule",
        var=f"table_1c_L1_previous_year",
        value=agg_with_subtotals.to_dict(orient="records"),
        db_path=db_path,
        anchor=f"table_1c",
        gt_table=tbl,
    )

    logging.debug("Stored 1c table and data")

    return agg_with_subtotals
          
# Main execution
try:
    conn = sqlite3.connect(db_path)
    df = fetch_latest_table_data(conn, "c0_commitments_summa", cutoff)
    agg = build_commitment_detail_table_2(df, current_year, report, db_path)
except Exception as e:
    print("Error:", e)
    logging.error(f"Main execution error: {str(e)}")
finally:
    conn.close()

DEBUG:selenium.webdriver.common.selenium_manager:Selenium Manager binary found at: c:\Users\vinci\anaconda3\envs\qenv\Lib\site-packages\selenium\webdriver\common\windows\selenium-manager.exe
DEBUG:selenium.webdriver.common.selenium_manager:Executing process: c:\Users\vinci\anaconda3\envs\qenv\Lib\site-packages\selenium\webdriver\common\windows\selenium-manager.exe --browser chrome --debug --language-binding python --output json
DEBUG:selenium.webdriver.common.selenium_manager:Found chromedriver 138.0.7204.92 in PATH: f:\vinci\Projects\QuarterlyReport\chromedriver.exe
DEBUG:selenium.webdriver.common.selenium_manager:chrome detected at C:\Program Files\Google\Chrome\Application\chrome.exe
DEBUG:selenium.webdriver.common.selenium_manager:Running command: wmic datafile where name='C:\\Program Files\\Google\\Chrome\\Application\\chrome.exe' get Version /value
DEBUG:selenium.webdriver.common.selenium_manager:Output: "\r\r\n\r\r\nVersion=138.0.7204.97\r\r\n\r\r\n\r\r\n\r"
DEBUG:selenium.webdr

In [7]:
import sqlite3
import json
import os
import logging
import pandas as pd
from datetime import datetime
from io import BytesIO
from great_tables import GT, loc, style, px, html
from great_tables.data import islands
from pathlib import Path
from typing import Any

# Define colors
BLUE = "#004A99"
LIGHT_BLUE = "#d6e6f4"
GRID_CLR = "#004A99"
DARK_BLUE = "#01244B"
DARK_GREY = '#242425'

# Parameters
db_path = "database/reporting.db"
cutoff = pd.to_datetime("2025-04-15")
current_year = 2025
report = "Quarterly_Report"

# Configure logging
logging.basicConfig(level=logging.DEBUG)

def insert_variable(
    report: str,
    module: str,
    var: str,
    value: Any,
    db_path: str,
    anchor: str | None = None,
    gt_table=None,
) -> None:
    """
    Overwrite the row (report_name, var_name) with a new value (and picture).

    Exactly ONE row per variable is kept.
    """
    con = sqlite3.connect(db_path)
    cur = con.cursor()
    try:
        # 1) remove any previous copy of this variable
        cur.execute(
            "DELETE FROM report_variables WHERE report_name = ? AND var_name = ?",
            (report, var),
        )

        # 2) serialise the Python value
        val_json = json.dumps(value, default=str)

        # 3) optional: render great‑tables object to PNG → bytes
        gt_image = None
        if gt_table is not None:
            tmp = Path(f"__gt_{var}.png")
            gt_table.save(tmp)  # playwright renders PNG
            gt_image = tmp.read_bytes()
            tmp.unlink(missing_ok=True)

        # 4) insert the fresh row
        cur.execute(
            """
            INSERT INTO report_variables
                  (report_name, module_name, var_name,
                   anchor_name, value, gt_image, created_at)
            VALUES (?, ?, ?, ?, ?, ?, CURRENT_TIMESTAMP)
            """,
            (report, module, var, anchor or var, val_json, gt_image),
        )

        con.commit()
        logging.debug("Stored variable %s for report %s (rowid=%s)",
                      var, report, cur.lastrowid)

    except Exception as exc:
        con.rollback()
        logging.error("insert_variable failed for %s/%s: %s", report, var, exc)
        raise
    finally:
        con.close()

# Fetch latest table data
def fetch_latest_table_data(conn: sqlite3.Connection, table_alias: str, cutoff: pd.Timestamp) -> pd.DataFrame:
    cutoff_str = cutoff.isoformat()
    query = """
        SELECT uploaded_at, id
        FROM upload_log
        WHERE table_alias = ?
        ORDER BY ABS(strftime('%s', uploaded_at) - strftime('%s', ?))
        LIMIT 1
    """
    result = conn.execute(query, (table_alias, cutoff_str)).fetchone()
    if not result:
        raise ValueError(f"No uploads found for table alias '{table_alias}' near cutoff {cutoff_str}")
    closest_uploaded_at, upload_id = result
    df = pd.read_sql_query(
        f"SELECT * FROM {table_alias} WHERE upload_id = ?",
        conn,
        params=(upload_id,)
    )
    return df

def build_commitment_detail_table_1(df: pd.DataFrame, current_year: int, report: str, db_path: str) -> pd.DataFrame:
    df["FR ILC Date (dd/mm/yyyy)"] = pd.to_datetime(df["FR ILC Date (dd/mm/yyyy)"], errors="coerce")

    next_year = current_year + 1
    eoy_next = pd.Timestamp(f"{next_year}-12-31")
    eoy_this = pd.Timestamp(f"{current_year}-12-31")

    # Budget‑type mapping helper
    def map_budget_type(val):
        if pd.isna(val):
            return None
        v = str(val).upper()
        if "VOBU" in v: return "VOBU/EFTA/IAR2/2"
        if "EFTA" in v: return "VOBU/EFTA/IAR2/2"
        if "IAR2/2" in v: return "VOBU/EFTA/IAR2/2"
        return val

    df.loc[:, 'Fund Source'] = df['Fund Source'].apply(map_budget_type)

    
    def map_fund_type(val):
            if pd.isna(val):
                return None
            v = str(val).strip().upper()
            if "GLOBAL" in v and "COMMITMENT" in v:
                return "MainCalls"
            if "PROVISIONAL" in v and "COMMITMENT" in v:
                return "Experts"
            return val

    df.loc[:, 'Source_Type'] = df['FR Earmarked Document Type Desc'].apply(map_fund_type)

    # ---------- GLOBAL COMMITMENTS -------------------------------------
    global_df = df[(df["Source_Type"] == "MainCalls") &
                   (df["FR ILC Date (dd/mm/yyyy)"] == eoy_next) & (df['Fund Source'] == "VOBU/EFTA/IAR2/2")].copy()

    logging.debug(f"Global Commitments - Rows after filter: {len(global_df)}")

    global_df = global_df.rename(columns={
        "FR Accepted Amount": "L1_Commitment_or_Direct_L2_1",
        "FR Consumption by PO Amount": "L2_Commitment_or_Payment_2"
    })

    agg_global = (global_df
                  .groupby(["FR Fund Reservation Desc", "Fund Source"],
                           as_index=False)[
                      ["L1_Commitment_or_Direct_L2_1", "L2_Commitment_or_Payment_2"]]
                  .sum())

    agg_global["RAC_on_L1_Commitment_or_RAL_Direct_L2_3"] = agg_global["L1_Commitment_or_Direct_L2_1"] + agg_global["L2_Commitment_or_Payment_2"]
    agg_global["Commitment_Implementation_rate_4"] = agg_global["L2_Commitment_or_Payment_2"] / (-1 * agg_global["L1_Commitment_or_Direct_L2_1"])

    global_df = agg_global[[
        "FR Fund Reservation Desc", "Fund Source", "L1_Commitment_or_Direct_L2_1", "L2_Commitment_or_Payment_2",
        "RAC_on_L1_Commitment_or_RAL_Direct_L2_3", "Commitment_Implementation_rate_4"
    ]]

    global_cols = [
        "FR Fund Reservation Desc", "Fund Source", "L1_Commitment_or_Direct_L2_1", "L2_Commitment_or_Payment_2",
        "RAC_on_L1_Commitment_or_RAL_Direct_L2_3", "Commitment_Implementation_rate_4"
    ]

    # ---------- PROVISIONAL COMMITMENTS -------------------------------------
    prov_df = df[(df["Source_Type"] == "Experts") &
                    (df["FR ILC Date (dd/mm/yyyy)"] ==  eoy_this)  & (df['Fund Source'] == "VOBU/EFTA/IAR2/2")].copy()

    logging.debug(f"Provisional Commitments - Rows after filter: {len(prov_df)}")

    prov_df = prov_df.rename(columns={
        "FR Accepted Amount": "L1_Commitment_or_Direct_L2_1",
        "FR Consumption by Payment Amount": "L2_Commitment_or_Payment_2"
    })

    prov_df["FR Fund Reservation Desc"] = 'Experts'

    agg_prov = (prov_df
                .groupby(["FR Fund Reservation Desc", "Fund Source"],
                         as_index=False)[
                    ["L1_Commitment_or_Direct_L2_1", "L2_Commitment_or_Payment_2"]]
                .sum())

    agg_prov["RAC_on_L1_Commitment_or_RAL_Direct_L2_3"] = agg_prov["L1_Commitment_or_Direct_L2_1"] + agg_prov["L2_Commitment_or_Payment_2"]
    agg_prov["Commitment_Implementation_rate_4"] = agg_prov["L2_Commitment_or_Payment_2"] / (-1 * agg_prov["L1_Commitment_or_Direct_L2_1"])

    prov_df = agg_prov[[
        "FR Fund Reservation Desc", "Fund Source", "L1_Commitment_or_Direct_L2_1", "L2_Commitment_or_Payment_2",
        "RAC_on_L1_Commitment_or_RAL_Direct_L2_3", "Commitment_Implementation_rate_4"
    ]]

    prov_cols = [
        "FR Fund Reservation Desc", "Fund Source", "L1_Commitment_or_Direct_L2_1", "L2_Commitment_or_Payment_2",
        "RAC_on_L1_Commitment_or_RAL_Direct_L2_3", "Commitment_Implementation_rate_4"
    ]

    # Check if both DataFrames are empty
    if global_df.empty and prov_df.empty:
        logging.warning("Both Global and Provisional Commitment DataFrames are empty. Returning an empty DataFrame.")
        # Create an empty DataFrame with the correct structure
        empty_df = pd.DataFrame(columns=global_cols)
        return empty_df

    # Prepare DataFrames for concatenation
    concat_dfs = []
    if not global_df.empty:
        concat_dfs.append(global_df[global_cols])
    if not prov_df.empty:
        concat_dfs.append(prov_df[prov_cols])

    # Concatenate only if there are DataFrames to concatenate
    if concat_dfs:
        combined = pd.concat(concat_dfs, axis=0, ignore_index=True)
    else:
        # This case should already be handled above, but adding for completeness
        combined = pd.DataFrame(columns=global_cols)

    # -------- Add subtotals by Fund Source -----------------------------------
    if not combined.empty:
        # Group by Fund Source and compute subtotals
        subtotals = combined.groupby("Fund Source", as_index=False).agg({
            "L1_Commitment_or_Direct_L2_1": "sum",
            "L2_Commitment_or_Payment_2": "sum",
            "RAC_on_L1_Commitment_or_RAL_Direct_L2_3": "sum",
        })

        # Add the ratio for subtotals
        subtotals["Commitment_Implementation_rate_4"] = subtotals["L2_Commitment_or_Payment_2"] / (-1 * subtotals["L1_Commitment_or_Direct_L2_1"])
        subtotals["FR Fund Reservation Desc"] = "Subtotal"

        # Concatenate original data with subtotals
        final_rows = []
        for fund_source in combined["Fund Source"].unique():
            # Rows for this Fund Source
            group_rows = combined[combined["Fund Source"] == fund_source].copy()
            final_rows.append(group_rows)
            # Corresponding subtotal row
            subtotal_row = subtotals[subtotals["Fund Source"] == fund_source].copy()
            final_rows.append(subtotal_row)

        # Combine all rows into the final DataFrame
        agg_with_subtotals = pd.concat(final_rows, ignore_index=True)
    else:
        agg_with_subtotals = combined  # Empty DataFrame with correct columns

    # -------- GreatTables object -------------------------------------------
    # Only create the table if there is data
    if not agg_with_subtotals.empty:
        tbl = (
            GT(
                agg_with_subtotals,
                rowname_col="FR Fund Reservation Desc",
                groupname_col="Fund Source"
            )
            .tab_header(
                title="HE"
            )
            .tab_style(
                style.text(color=DARK_BLUE, weight="bold", align="center", font='Arial'),
                locations=loc.header()
            )
            .tab_stubhead(label="Commitment Type (L1 or L2 Direct)")
            .tab_style(
                style=[
                    style.text(color=DARK_BLUE, weight="bold", font='Arial'),
                    style.fill(color=LIGHT_BLUE),
                    style.css(f"border-bottom: 2px solid {DARK_BLUE}; border-right: 2px solid {DARK_BLUE}; border-top: 2px solid {DARK_BLUE}; border-left: 2px solid {DARK_BLUE}"),
                    style.css("max-width:200px; line-height:1.2"),
                ],
                locations=loc.row_groups()
            )
            .fmt_number(columns=[
                "L1_Commitment_or_Direct_L2_1",
                "L2_Commitment_or_Payment_2",
                "RAC_on_L1_Commitment_or_RAL_Direct_L2_3"
            ], accounting=True, decimals=2)
            .fmt_percent(
                columns="Commitment_Implementation_rate_4",
                decimals=2
            )
            .cols_label(
                L1_Commitment_or_Direct_L2_1=html("L1 Commitment or Direct L2<br>(1)"),
                L2_Commitment_or_Payment_2=html("L2 Commitment or Payment<br>(2)"),
                RAC_on_L1_Commitment_or_RAL_Direct_L2_3=html("RAC on L1 Commitment or RAL Direct L2<br>(3) = (1) + (2)"),
                Commitment_Implementation_rate_4=html("% Commitment Implementation Rate<br>(4) = (2) / (1)")
            )
            .opt_table_font(font="Arial")
            .tab_style(
                style=[
                    style.fill(color=BLUE),
                    style.text(color="white", weight="bold", align="center"),
                    style.css("max-width:200px; line-height:1.2")
                ],
                locations=loc.column_labels()
            )
            .tab_style(
                style=[
                    style.fill(color=BLUE),
                    style.text(color="white", weight="bold", align="center"),
                    style.css("text-align: center; vertical-align: middle; max-width:200px; line-height:1.2")
                ],
                locations=loc.stubhead()
            )
            .tab_style(
                style=style.borders(weight="1px", color=DARK_BLUE),
                locations=loc.stub()
            )
            .tab_style(
                style=style.borders(sides="all", color=DARK_BLUE, weight="1px"),
                locations=loc.body()
            )
            .tab_style(
                style=style.borders(color=DARK_BLUE, weight="2px"),
                locations=[loc.column_labels(), loc.stubhead()]
            )
            .tab_style(
                style=[style.fill(color="#E6E6FA"), style.text(color="black", weight="bold")],
                locations=loc.body(rows=agg_with_subtotals.index[agg_with_subtotals["FR Fund Reservation Desc"] == "Subtotal"].tolist())
            )
            .tab_options(table_body_border_bottom_color=DARK_BLUE,
                         table_body_border_bottom_width="2px")
            .tab_options(table_border_right_color=DARK_BLUE,
                         table_border_right_width="2px")
            .tab_options(table_border_left_color=DARK_BLUE,
                         table_border_left_width="2px")
            .tab_options(table_border_top_color=DARK_BLUE,
                         table_border_top_width="2px")
            .tab_options(column_labels_border_top_color=DARK_BLUE,
                         column_labels_border_top_width="2px")
            .tab_source_note("Source: Summa DataWarehouse")
            .tab_source_note("BO Report: C0_COMMITMENTS_SUMMA")
        )

        # -------- store --------------------------------------------------------
        insert_variable(
            report=report,
            module="BudgetModule",
            var=f"table_1b_L1_current_year",
            value=agg_with_subtotals.to_dict(orient="records"),
            db_path=db_path,
            anchor=f"table_1b",
            gt_table=tbl,
        )

        logging.debug("Stored 1b table and data")
    else:
        logging.debug("No data to store for table 1b (empty DataFrame).")

    return agg_with_subtotals

# Main execution
try:
    conn = sqlite3.connect(db_path)
    df = fetch_latest_table_data(conn, "c0_commitments_summa", cutoff)
    agg = build_commitment_detail_table_1(df, current_year, report, db_path)
except Exception as e:
    print("Error:", e)
    logging.error(f"Main execution error: {str(e)}")
finally:
    conn.close()

DEBUG:root:Global Commitments - Rows after filter: 0
DEBUG:root:Provisional Commitments - Rows after filter: 6
DEBUG:selenium.webdriver.common.selenium_manager:Selenium Manager binary found at: c:\Users\vinci\anaconda3\envs\qenv\Lib\site-packages\selenium\webdriver\common\windows\selenium-manager.exe
DEBUG:selenium.webdriver.common.selenium_manager:Executing process: c:\Users\vinci\anaconda3\envs\qenv\Lib\site-packages\selenium\webdriver\common\windows\selenium-manager.exe --browser chrome --debug --language-binding python --output json
DEBUG:selenium.webdriver.common.selenium_manager:Found chromedriver 138.0.7204.92 in PATH: f:\vinci\Projects\QuarterlyReport\chromedriver.exe
DEBUG:selenium.webdriver.common.selenium_manager:chrome detected at C:\Program Files\Google\Chrome\Application\chrome.exe
DEBUG:selenium.webdriver.common.selenium_manager:Running command: wmic datafile where name='C:\\Program Files\\Google\\Chrome\\Application\\chrome.exe' get Version /value
DEBUG:selenium.webdriv

In [8]:
cutoff = pd.to_datetime("2025-04-15")

try:
    conn = sqlite3.connect(db_path)
    df = fetch_latest_table_data(conn, "c0_commitments_summa", cutoff)
except Exception as e:
    print("Error:", e)
    logging.error(f"Main execution error: {str(e)}")
finally:
    conn.close()
    
# Define expected columns for validation
expected_columns = [
    "FR ILC Date (dd/mm/yyyy)", "Fund Source", "FR Earmarked Document Type Desc",
    "FR Accepted Amount", "FR Consumption by PO Amount"
]

# Validate input DataFrame
if not isinstance(df, pd.DataFrame):
    logging.error("Input 'df' is not a pandas DataFrame.")

if df.empty:
    logging.warning("Input DataFrame is empty. Returning empty DataFrame with expected columns.")
   
# Check for missing columns
missing_columns = [col for col in expected_columns if col not in df.columns]
if missing_columns:
    logging.error(f"Missing required columns: {missing_columns}")
  

# Log input DataFrame info
logging.debug(f"Input DataFrame shape: {df.shape}")
logging.debug(f"Input DataFrame columns: {df.columns.tolist()}")

# logging.debug(f"Raw date values (first 5): {df['FR ILC Date (dd/mm/yyyy)'].head().tolist()}")
# date_formats = ["%d/%m/%Y", "%Y-%m-%d", "%m/%d/%Y", "%d-%m-%Y"]  # Added %d-%m-%Y
# df["FR ILC Date (dd/mm/yyyy)"] = None
# for fmt in date_formats:
#     try:
#         df["FR ILC Date (dd/mm/yyyy)"] = pd.to_datetime(
#             df["FR ILC Date (dd/mm/yyyy)"], format=fmt, errors="coerce"
#         )
#         if df["FR ILC Date (dd/mm/yyyy)"].notna().sum() > 0:
#             logging.debug(f"Successfully parsed dates using format: {fmt}")
#             break
#     except Exception as e:
#         logging.debug(f"Date format {fmt} failed: {str(e)}")

# # If still no valid dates, log and proceed with unfiltered data as a fallback
# if df["FR ILC Date (dd/mm/yyyy)"].isna().all():
#     logging.warning("No dates parsed with specified formats. Proceeding without date filter.")
#     df["FR ILC Date (dd/mm/yyyy)"] = pd.to_datetime(df["FR ILC Date (dd/mm/yyyy)"], errors="coerce")
# df["FR ILC Date (dd/mm/yyyy)"] = pd.to_datetime(df["FR ILC Date (dd/mm/yyyy)"], errors="coerce", format="%d/%m/%Y")
# logging.debug(f"Rows with valid dates: {df['FR ILC Date (dd/mm/yyyy)'].notna().sum()}")
eoy_this = pd.Timestamp(f"{current_year}-12-31")

# Budget-type mapping helper
def map_budget_type(val):
    if pd.isna(val):
        return None
    v = str(val).upper()
    if "VOBU" in v:
        return "VOBU/EFTA/IAR2/2"
    if "EFTA" in v:
        return "VOBU/EFTA/IAR2/2"
    if "IAR2/2" in v:
        return "VOBU/EFTA/IAR2/2"
    return val

df["Fund Source"] = df["Fund Source"].apply(map_budget_type)
logging.debug(f"Unique Fund Source values after mapping: {df['Fund Source'].unique()}")

# Filter the DataFrame
filtered = df[
    (df["FR Earmarked Document Type Desc"] == "Global Commitment") &
    (df["FR ILC Date (dd/mm/yyyy)"] == eoy_this)
].copy()

logging.debug(f"Rows after filtering for Global Commitments: {len(filtered)}")

if filtered.empty:
    logging.warning("Filtered DataFrame is empty after applying filters. Returning empty DataFrame.")


# Rename columns
filtered = filtered.rename(columns={
    "FR Accepted Amount": "L1_Commitment_1",
    "FR Consumption by PO Amount": "L2_Commitment_2"
})

filtered

DEBUG:root:Input DataFrame shape: (124, 28)
DEBUG:root:Input DataFrame columns: ['Financial Management Area', 'Commitment Item', 'Official Item', 'Fund Source', 'Budget Period', 'Funded Program', 'FR Year Of Origin', 'FR Fund Reservation Key', 'FR Earmarked Document Type', 'FR Earmarked Document Type Desc', 'FR ILC Date (dd/mm/yyyy)', 'FR Workflow Status', 'FR Workflow Last IA Action Date', 'FR Workflow Last IA Person Id', 'FR Workflow Last VA Action Date', 'FR Workflow Last VA Person Id', 'FR Workflow Last AOS Action Date', 'FR Workflow Last AOS Person Id', 'FR Fund Reservation Desc', 'FR Final Date of Implementation', 'FR Counted in Budget Implementation (COM)', 'FR Accepted Amount', 'FR Consumption by PO Amount', 'FR Consumption by Payment Amount', 'FR Open Amount - RAL - Payments Made (PD Approved)', 'FR Open Amount - RAL - Cost Accepted (Invoice Posted)', 'upload_id', 'uploaded_at']
DEBUG:root:Unique Fund Source values after mapping: ['EARN/N' 'VOBU/EFTA/IAR2/2']
DEBUG:root:Rows a

,Financial Management Area,Commitment Item,Official Item,Fund Source,Budget Period,Funded Program,FR Year Of Origin,FR Fund Reservation Key,FR Earmarked Document Type,FR Earmarked Document Type Desc,...,FR Fund Reservation Desc,FR Final Date of Implementation,FR Counted in Budget Implementation (COM),L1_Commitment_1,L2_Commitment_2,FR Consumption by Payment Amount,FR Open Amount - RAL - Payments Made (PD Approved),FR Open Amount - RAL - Cost Accepted (Invoice Posted),upload_id,uploaded_at


In [8]:
try:
        # Convert date column to datetime, handling invalid dates
        # df["FR ILC Date (dd/mm/yyyy)"] = pd.to_datetime(df["FR ILC Date (dd/mm/yyyy)"], errors="coerce", format="%d/%m/%Y")
        # logging.debug(f"Rows with valid dates: {df['FR ILC Date (dd/mm/yyyy)'].notna().sum()}")
        logging.debug(f"Raw date values (first 5): {df['FR ILC Date (dd/mm/yyyy)'].head().tolist()}")
        date_formats = ["%d/%m/%Y", "%Y-%m-%d", "%m/%d/%Y", "%d-%m-%Y"]  # Added %d-%m-%Y
        df["FR ILC Date (dd/mm/yyyy)"] = None
        for fmt in date_formats:
            try:
                df["FR ILC Date (dd/mm/yyyy)"] = pd.to_datetime(
                    df["FR ILC Date (dd/mm/yyyy)"], format=fmt, errors="coerce"
                )
                if df["FR ILC Date (dd/mm/yyyy)"].notna().sum() > 0:
                    logging.debug(f"Successfully parsed dates using format: {fmt}")
                    break
            except Exception as e:
                logging.debug(f"Date format {fmt} failed: {str(e)}")

        # If still no valid dates, log and proceed with unfiltered data as a fallback
        if df["FR ILC Date (dd/mm/yyyy)"].isna().all():
            logging.warning("No dates parsed with specified formats. Proceeding without date filter.")
            df["FR ILC Date (dd/mm/yyyy)"] = pd.to_datetime(df["FR ILC Date (dd/mm/yyyy)"], errors="coerce")

        eoy_this = pd.Timestamp(f"{current_year}-12-31")

        # Budget-type mapping helper
        def map_budget_type(val):
            if pd.isna(val):
                return None
            v = str(val).upper()
            if "VOBU" in v:
                return "VOBU/EFTA/IAR2/2"
            if "EFTA" in v:
                return "VOBU/EFTA/IAR2/2"
            if "IAR2/2" in v:
                return "VOBU/EFTA/IAR2/2"
            return val

        df["Fund Source"] = df["Fund Source"].apply(map_budget_type)
        logging.debug(f"Unique Fund Source values after mapping: {df['Fund Source'].unique()}")

        # Filter the DataFrame
        filtered = df[
            (df["FR Earmarked Document Type Desc"] == "Global Commitment") &
            (df["FR ILC Date (dd/mm/yyyy)"] == eoy_this)
        ].copy()

        logging.debug(f"Rows after filtering for Global Commitments: {len(filtered)}")

        if filtered.empty:
            logging.warning("Filtered DataFrame is empty after applying filters. Returning empty DataFrame.")


        # Rename columns
        filtered = filtered.rename(columns={
            "FR Accepted Amount": "L1_Commitment_1",
            "FR Consumption by PO Amount": "L2_Commitment_2"
        })

        # Aggregation
        agg = (filtered
               .groupby(["FR Fund Reservation Desc", "Fund Source"],
                        as_index=False)[
                   ["L1_Commitment_1", "L2_Commitment_2"]
               ].sum())
        logging.debug(f"Aggregated DataFrame shape: {agg.shape}")

        # Compute additional columns with zero division handling
        agg["RAC_on_L1_Commitment_3"] = agg["L1_Commitment_1"] + agg["L2_Commitment_2"]
        agg["ratio_L2_on_L1_Commitment_4"] = (
            agg["L2_Commitment_2"] / (-1 * agg["L1_Commitment_1"]).replace(0, pd.NA)
        ).fillna(0)  # Handle division by zero

        agg = agg[[
            "FR Fund Reservation Desc", "Fund Source", "L1_Commitment_1", "L2_Commitment_2",
            "RAC_on_L1_Commitment_3", "ratio_L2_on_L1_Commitment_4"
        ]]

        # Add subtotals by Fund Source
        try:
            subtotals = agg.groupby("Fund Source", as_index=False).agg({
                "L1_Commitment_1": "sum",
                "L2_Commitment_2": "sum",
                "RAC_on_L1_Commitment_3": "sum",
            })

            subtotals["ratio_L2_on_L1_Commitment_4"] = (
                subtotals["L2_Commitment_2"] / (-1 * subtotals["L1_Commitment_1"]).replace(0, pd.NA)
            ).fillna(0)
            subtotals["FR Fund Reservation Desc"] = "Subtotal"

            final_rows = []
            for fund_source in agg["Fund Source"].unique():
                group_rows = agg[agg["Fund Source"] == fund_source].copy()
                final_rows.append(group_rows)
                subtotal_row = subtotals[subtotals["Fund Source"] == fund_source].copy()
                final_rows.append(subtotal_row)

            agg_with_subtotals = pd.concat(final_rows, ignore_index=True)
            logging.debug(f"Final DataFrame with subtotals shape: {agg_with_subtotals.shape}")
        except Exception as e:
            logging.error(f"Error adding subtotals: {str(e)}")
            agg_with_subtotals = agg  # Fall back to aggregated DataFrame without subtotals

        # GreatTables object
        if not agg_with_subtotals.empty:
            try:
                tbl = (
                    GT(
                        agg_with_subtotals,
                        rowname_col="FR Fund Reservation Desc",
                        groupname_col="Fund Source"
                    )
                    .tab_stubhead(label="L1 Commitments")
                    .tab_style(
                        style=[
                            style.text(color="white", weight="bold"),
                            style.fill(color=LIGHT_BLUE),
                            style.css(f"border-bottom: 2px solid {DARK_BLUE}; border-right: 2px solid {DARK_BLUE}; "
                                      f"border-top: 2px solid {DARK_BLUE}; border-left: 2px solid {DARK_BLUE}"),
                            style.css("max-width:200px; line-height:1.2"),
                        ],
                        locations=loc.row_groups()
                    )
                    .fmt_number(columns=[
                        "L1_Commitment_1",
                        "L2_Commitment_2",
                        "RAC_on_L1_Commitment_3"
                    ], accounting=True, decimals=2)
                    .fmt_percent(
                        columns="ratio_L2_on_L1_Commitment_4",
                        decimals=2
                    )
                    .cols_label(
                        L1_Commitment_1=html("L1 Commitment<br>(1)"),
                        L2_Commitment_2=html("L2 Commitments<br>(2)"),
                        RAC_on_L1_Commitment_3=html("RAC on L1 Commitment <br>(3) = (1) - (2)"),
                        ratio_L2_on_L1_Commitment_4=html("% L1 consumed by L2 (indirect)<br> (4) = (2) / (1)")
                    )
                    .opt_table_font(font="Arial")
                    .tab_style(
                        style=[style.fill(color=BLUE),
                               style.text(color="white", weight="bold", align="center"),
                               style.css("max-width:200px; line-height:1.2")],
                        locations=loc.column_labels()
                    )
                    .tab_style(
                        style=[style.fill(color=BLUE),
                               style.text(color="white", weight="bold"),
                               style.css("text-align: center; vertical-align: middle; max-width:200px; line-height:1.2")],
                        locations=loc.stubhead()
                    )
                    .tab_style(
                        style=style.borders(weight="1px", color=DARK_BLUE),
                        locations=loc.stub()
                    )
                    .tab_style(
                        style=style.borders(sides="all", color=DARK_BLUE, weight="1px"),
                        locations=loc.body()
                    )
                    .tab_style(
                        style=style.borders(color=DARK_BLUE, weight="2px"),
                        locations=[loc.column_labels(), loc.stubhead()]
                    )
                    .tab_style(
                        style=[style.fill(color="#E6E6FA"), style.text(color="black", weight="bold")],
                        locations=loc.body(rows=agg_with_subtotals.index[agg_with_subtotals["FR Fund Reservation Desc"] == "Subtotal"].tolist())
                    )
                    .tab_style(
                        style=[style.fill(color="#E6E6FA"), style.text(weight="bold")],
                        locations=loc.stub(rows=[-1])
                    )
                    .tab_options(table_body_border_bottom_color=DARK_BLUE, table_body_border_bottom_width="2px")
                    .tab_options(table_border_right_color=DARK_BLUE, table_border_right_width="2px")
                    .tab_options(table_border_left_color=DARK_BLUE, table_border_left_width="2px")
                    .tab_options(table_border_top_color=DARK_BLUE, table_border_top_width="2px")
                    .tab_options(column_labels_border_top_color=DARK_BLUE, column_labels_border_top_width="2px")
                    .tab_source_note("Source: Summa DataWarehouse")
                    .tab_source_note("BO Report: C0_COMMITMENTS_SUMMA")
                )
            except Exception as e:
                logging.error(f"Error building GreatTables object: {str(e)}")
                

            # Store the table
            try:
                insert_variable(
                    report=report,
                    module="BudgetModule",
                    var=f"table_1c_L1_previous_year",
                    value=agg_with_subtotals.to_dict(orient="records"),
                    db_path=db_path,
                    anchor=f"table_1c",
                    gt_table=tbl
                )
                logging.debug("Stored table_1c_L1_previous_year")
            except Exception as e:
                logging.error(f"Error storing table: {str(e)}")
                # Continue even if storage fails, as the table is already built
        else:
            logging.debug("No data to store for table_1c (empty DataFrame).")



except Exception as e:
    logging.error(f"Unexpected error in build_commitment_detail_table_2: {str(e)}")

    

DEBUG:root:Raw date values (first 5): ['2025-12-31 00:00:00', '2025-12-31 00:00:00', '2025-12-31 00:00:00', '2025-12-31 00:00:00', '2025-12-31 00:00:00']
DEBUG:root:Unique Fund Source values after mapping: ['EARN/N' 'VOBU/EFTA/IAR2/2']
DEBUG:root:Rows after filtering for Global Commitments: 0
DEBUG:root:Aggregated DataFrame shape: (0, 4)
ERROR:root:Error adding subtotals: No objects to concatenate
DEBUG:root:No data to store for table_1c (empty DataFrame).


In [9]:
filtered

,Financial Management Area,Commitment Item,Official Item,Fund Source,Budget Period,Fund Management Center,Funded Program,FR Year Of Origin,FR Fund Reservation Key,FR Earmarked Document Type,...,FR Fund Reservation Desc,FR Final Date of Implementation,FR Counted in Budget Implementation (COM),L1_Commitment_1,L2_Commitment_2,FR Consumption by Payment Amount,FR Open Amount - RAL - Payments Made (PD Approved),FR Open Amount - RAL - Cost Accepted (Invoice Posted),upload_id,uploaded_at
